In [1]:
!pip install rouge-score
!pip install sacremoses

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=86f769b41ac627a1520caa7882433e2bd2327e6c5efc04dff6541afdcdf7638e
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 18.4 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=4bd65b96bcc0ee1e8bb9b5d72181d589fae63dbced555503d4a365e3703a8054
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [3]:
from transformers import BioGptTokenizer,BioGptForCausalLM
# Instantiating the model and tokenizer with gpt
tokenizer = BioGptTokenizer.from_pretrained('microsoft/biogpt')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model = BioGptForCausalLM.from_pretrained('microsoft/biogpt')

In [6]:
import pandas as pd

df = pd.read_excel('/kaggle/input/pubmedabstractcustom/output.xlsx',header=None)
df.columns = ['ID', 'Abstract']

In [15]:
print(df['Abstract'].iloc[:])

0    Gestational diabetes mellitus (GDM) is the mos...
1    Cardiovascular disease (CVD) is the leading ca...
2    Objective: Development of these guidelines is ...
3    Background: Although the body of evidence indi...
4    Non-alcoholic fatty liver disease (NAFLD) is t...
5    Non-alcoholic fatty liver disease (NAFLD) has ...
6    The metabolic syndrome (MetS) comprises indivi...
7    Two-thirds of patients with type 2 diabetes me...
8    Although consumption of sugar-sweetened bevera...
9    Clinical characteristics: Alström syndrome is ...
Name: Abstract, dtype: object


In [27]:
GPT_summarized_lists={}

for index, row in df.iterrows():
    inputs = tokenizer.encode_plus(row['Abstract'], return_tensors='pt', truncation=True, max_length=512)
    input_ids, attention_mask = inputs['input_ids'], inputs['attention_mask']

    # Generate summary
    summary_ids = model.generate(input_ids, attention_mask=attention_mask, min_length=90, max_length=512)

    GPT_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    print(str(row['ID']) + ': ' + GPT_summary)
    GPT_summarized_lists[row['ID']]=GPT_summary

# for index, row in df.iterrows():
#     input_text = "Summarize the text: "+ row['Abstract']
#     inputs = tokenizer.encode_plus(input_text, return_tensors='pt', truncation=True, padding='longest', max_length=512)
#     input_ids, attention_mask = inputs['input_ids'], inputs['attention_mask']

#     summary_ids = model.generate(input_ids, attention_mask=attention_mask, early_stopping=True, min_length=90, max_length=150, pad_token_id=tokenizer.eos_token_id)

#     GPT_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
#     print(str(row['ID'])+ ': ' + GPT_summary)
#     GPT_summarized_lists.append(GPT_summary)


30297319: Gestational diabetes mellitus (GDM) is the most common metabolic disturbance during pregnancy. The prevalence is rising and correlates with the increase in maternal obesity over recent decades. The etiology of GDM is complex, with genetic and environmental factors implicated in mechanistic and epidemiological studies. GDM begets important short- and long-term health risks for the mother, developing fetus, and offspring. This includes the high likelihood of subsequent maternal type 2 diabetes (T2DM), and possible adverse cardiometabolic phenotypes in the offspring. The most clinically and cost-effective methods of screening for GDM remain uncertain. Whilst treatments with lifestyle and pharmacological interventions have demonstrated short-term benefits, the long-term impact for the offspring of intrauterine exposure to antidiabetic medication remains unclear.
29669543: Cardiovascular disease (CVD) is the leading cause of mortality in people with type 2 diabetes mellitus (T2DM)

Input length of input_ids is 512, but `max_length` is set to 512. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


29704920: Two-thirds of patients with type 2 diabetes mellitus (T2DM) have arterial hypertension. Hypertension increases the incidence of both micro- and macrovascular complications in these patients, while the co-existence of these two major risk factors leads to a four-fold increased risk for cardiovascular disease (CVD) compared with normotensive non-diabetic controls. The aim of this article is to comprehensively review the literature and present updated information on targets for blood pressure (BP) and on the management of hypertension in patients with T2DM. A BP target of < 140 / 90 mmHg applies to most patients, but individualization is always important. All classes of antihypertensive drugs can be used in the management of hypertension in patients with T2DM, as long as they are effective and safe and after taking co-morbidities into account. Angiotensin-converting-enzyme (ACE) inhibitors or angiotensin receptor blockers (ARBs) are the ideal choice for initial or early treatmen

In [29]:
df_summaries = pd.DataFrame(list(GPT_summarized_lists.items()), columns=['ID', 'Summary'])

df_summaries.to_csv('summaries.csv', index=False)

In [49]:
from rouge_score import rouge_scorer

def compute_rouge(reference, candidate):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True)
    scores = scorer.score(reference, candidate)
    return scores

if __name__ == '__main__':
    file_path = "/kaggle/input/full-texts/Abstract-2529.txt"
    input_abstract = read_input_file(file_path)
    reference =  GPT_summary 
    candidate = input_abstract

    scores = compute_rouge(reference, candidate)
    for key, score in scores.items():
        print(f"{key.upper()}:")
        print(f"  Precision: {score.precision:.4f}")
        print(f"  Recall: {score.recall:.4f}")
        print(f"  F1 Score: {score.fmeasure:.4f}\n")


ROUGE1:
  Precision: 0.4606
  Recall: 0.4419
  F1 Score: 0.4510

ROUGE2:
  Precision: 0.1585
  Recall: 0.1520
  F1 Score: 0.1552

ROUGEL:
  Precision: 0.1818
  Recall: 0.1744
  F1 Score: 0.1780

ROUGELSUM:
  Precision: 0.1818
  Recall: 0.1744
  F1 Score: 0.1780

